In [44]:
#r "nuget: Plotly.NET;"
#r "nuget: Plotly.NET.ImageExport;"

using Plotly.NET;
using Plotly.NET.ImageExport;

double ema(double data, double prev_ema, int period, double attenuator = 0.0) {
    double k = 2.0 / (period + 1);
    return ((data*k) + (prev_ema*(1-k)))/(1-attenuator);
}

List<double> data = new() { 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0}; 
List<int> x = new() { 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16};
List<double> es = new();
List<double> e1 = new();
List<double> e2 = new();
List<double> e3 = new();

EMA is a convergent moving average where we don't need to keep a list of past values around; all that is needed to calculate the next value of EMA is a previous value, factor k [ k = 2.0/(period+1) ] and new value.

The core question is: how to START the EMA sequence?

The most direct way is to start the EMA sequence with an arbitrary number - either zero or the first value are commonly accepted 'igniters' of the calculation. Except both starting values are wrong: 
- starting EMA with the First Value creates overshooting EMA, requiring 50+ bars to converge to the correct series.
- starting EMA with a Zero undershoots the series, requiring 50+ bars to climb back to the correct series.
- starting EMA with a short SMA warming sequence at the beginning creates even more problems: transition from SMA to EMA is non-linear and results are not predictable.

But there is a better way.

I found an article on [David Owen's blog](https://blog.fugue88.ws/archives/2017-01/The-correct-way-to-start-an-Exponential-Moving-Average-EMA) that explains the math behind attenuation of early elements of EMA series.

Below is the simple comparison of four approaches:

- low EMA (series assumes that pre-value was 0)
- high EMA (series assumes that pre-value was same as the first value)
- SMA-EMA (series first calculates SMA for the duration of period and then uses the last SMA to calculate EMA)
- fixed EMA (uses diminishing attenuation to keep EMA between overshooting and undershooting EMA )


In [47]:
int period = 18;

double k = 2.0 / (period + 1);
double attenuator = ( 1 - k ) * 0.5;
double factor = 1;
double ema1 = 0;       // start too low
double ema2 = data[0]; // start too high
double ema3 = data[0]*(1-k-attenuator)/(1-k); 
double emas = data[0];
Console.WriteLine($"data\t low\t  high\t  s_ema   fixed\t");
for (int i=0; i<data.Count; i++) {
    factor *= attenuator;
    ema1 = ema(data[i], ema1, period); e1.Add(ema1);
    ema2 = ema(data[i], ema2, period); e2.Add(ema2);
    ema3 = ema(data[i], ema3, period); e3.Add(ema3/(1-factor));
    emas = (i<period)?data.GetRange(0, i+1).Average():ema(data[i], emas, period); es.Add(emas);
    Console.WriteLine($"{data[i]}\t {e1[i]:f3}\t  {e2[i]:f3}\t  {es[i]:f3}\t {e3[i]:f3}\t");
}

data	 low	  high	  s_ema   fixed	
1	 0.222	  1.000	  1.000	 1.000	
0	 0.173	  0.778	  0.500	 0.560	
0	 0.134	  0.605	  0.333	 0.393	
0	 0.105	  0.471	  0.250	 0.294	
0	 0.081	  0.366	  0.200	 0.226	
0	 0.063	  0.285	  0.167	 0.175	
1	 0.271	  0.444	  0.286	 0.358	
1	 0.433	  0.567	  0.375	 0.501	
1	 0.559	  0.663	  0.514	 0.611	
1	 0.657	  0.738	  0.622	 0.698	
1	 0.733	  0.796	  0.706	 0.765	
0	 0.570	  0.619	  0.549	 0.595	
0	 0.444	  0.482	  0.427	 0.463	
0	 0.345	  0.375	  0.332	 0.360	
0	 0.268	  0.291	  0.258	 0.280	
0	 0.209	  0.227	  0.201	 0.218	


Chart below shows different EMA calculations; note how 
- SMA-EMA line abruptly breaks when it switches from SMA to EMA
- high EMA and low EMA form a converging channel; the higher the period, the longer is the channel
- fixed EMA starts with the value[0] but immediately attenuates to the middle of the channel (between high and low EMA)

Feel free to play with parameters above (period and attenuator) and see how various EMAs react.

In [48]:
var d = Chart2D.Chart.Line<int,double,bool>(x, data, true, "Data").WithLineStyle(Width: 4, Color: Color.fromString("blue"));
var le = Chart2D.Chart.Line<int,double,bool>(x, es, true, "s_ema").WithLineStyle(Width: 2, Color: Color.fromString("green"));
var le1 = Chart2D.Chart.Line<int,double,bool>(x, e1, true, "low").WithLineStyle(Width: 2, Color: Color.fromString("red"));
var le2 = Chart2D.Chart.Line<int,double,bool>(x, e2, true, "high").WithLineStyle(Width: 2, Color: Color.fromString("red"));
var le3 = Chart2D.Chart.Line<int,double,bool>(x, e3, true, "fixed").WithLineStyle(Width: 3, Color: Color.fromString("purple"));
var chart = Chart.Combine(new []{d,le,le1,le2,le3})
    .WithSize(1200,600)
    .WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false))
    .WithXAxisRangeSlider(RangeSlider.init(Visible:false));

chart.SavePNG("ema_study");
chart

<!-- Plotly chart will be drawn inside this DIV -->

![EMA study chart](./ema_study.png)